In [1]:
import os
import pandas as pd
import numpy as np
import requests
import pandas as pd
from scipy.stats import hypergeom
from tqdm import tqdm
import gseapy as gp
currdir=os.getcwd()
parent = os.path.dirname(currdir)
gparent = os.path.dirname(parent)
parent

'/home/poojaparameswaran/Documents/SoderlingLab/SpatialProteomics'

In [3]:
KOGENE = 'LRRK2'
PATH_OF_INT = 'ParkinsonsDisease'
modules_file = f"{parent}/example_data/tables/{KOGENE}_moduleResults_KEGGortholog.csv" # reading from example_data
all_modules = pd.read_csv(modules_file)
symbol = all_modules[['Symbol', 'KEGG_Ortholog']]
symbol  

,Symbol,KEGG_Ortholog
0,Arfgef2,K18442
1,Ift140,K19672
2,Rgs7,K16449
3,Gpx4,K05361
4,H1f4,K11275
...,...,...
1952,Atxn10,K19323
1953,Arpc2,K05758
1954,Carm1,K05931
1955,Uso1,FAIL


In [10]:
def ensure_dirs_exists(path):
    if "." in path:
	    path = os.path.dirname(path)
    if not os.path.exists(path):
        os.makedirs(path)
    return

In [11]:
PATHWAY_NAME = "ParkinsonsDisease"
url = 'https://rest.kegg.jp/get/mmu05012' # get parkinsons 
response = requests.get(url)

# The response will contain the list of genes associated with the mmu05012 pathway
data = response.text

In [12]:
after_GENE = data.split("GENE")
Gene_and_Junk = after_GENE[-1]
GENEINFO = Gene_and_Junk.split("COMPOUND")[0]


In [13]:
lines = GENEINFO.strip().split('\n')

# Parse each line into structured data
data = []
for line in lines:
    parts = line.split('  ')
    parts = [p for p in parts if p]  # Remove empty strings

    if len(parts) >= 2:
        gene_id = parts[0].strip()
        description = parts[1].strip()
        gene_name, description = description.split(";")[0] , description.split(";")[-1]
        desc, KO = description.split("KO:")[0], description.split("KO:")[-1]
        KO = KO.split("]")[0]
        data.append({'Entrez': int(gene_id), 'Symbol': gene_name, "Description": desc,
                     "KEGG_Ortholog": KO})
    else:
        print(parts)
        data.append({'Entrez': int(parts.split(";")[0]), 'Symbol': parts.split(";")[-1]})

PDDF = pd.DataFrame(data)

PDDF.to_csv(f"{PATHWAY_NAME}genes.csv")
PDDF

,Entrez,Symbol,Description,KEGG_Ortholog
0,11540,Adora2a,adenosine A2a receptor [,K04266
1,14680,Gnal,"guanine nucleotide binding protein, alpha sti...",K04633
2,14683,Gnas,GNAS complex locus [,K04632
3,13489,Drd2,dopamine receptor D2 [,K04145
4,14677,Gnai1,G protein subunit alpha i1 [,K04630
...,...,...,...,...
261,26419,Mapk8,mitogen-activated protein kinase 8 [,K04440
262,26420,Mapk9,mitogen-activated protein kinase 9 [,K04440
263,26414,Mapk10,mitogen-activated protein kinase 10 [,K04440
264,20655,Sod1,"superoxide dismutase 1, soluble [",K04565


In [14]:
df_1 = all_modules.loc[all_modules['Membership'] == 2]
all_modules['KEGG_Ortholog'].dtype
# PDDF['KEGG ortholog'].dtype
all_modules

,Unnamed: 0,UniProt,Entrez,Symbol,Membership,KEGG_Ortholog
0,0,A2A5R2,99371,Arfgef2,0,K18442
1,1,E9PY46,106633,Ift140,0,K19672
2,2,O54829,24012,Rgs7,0,K16449
3,3,O70325,625249,Gpx4,0,K05361
4,4,P43274,50709,H1f4,0,K11275
...,...,...,...,...,...,...
1952,1952,P28658,54138,Atxn10,45,K19323
1953,1953,Q9CVB6,76709,Arpc2,45,K05758
1954,1954,Q9WVG6,59035,Carm1,45,K05931
1955,1955,Q9Z1Z0,56041,Uso1,45,FAIL


1. Calculate FoldEnrichment
2. Hypergeometric test to determine significancer of ovelrap. yielding P-val
3. Adjust p value for multiple testin (FDR) since doing multiple memberships
    - benjamini-hochberg procedure to control FDR.

In [8]:
def get_EnrichmentAnalysis(module_num, module_df,all_modules, KEGGpathwaydf, comparison='Symbol'):
    overlap = set(KEGGpathwaydf[comparison]).intersection(set(module_df[comparison]))
    overlap = list(KEGGpathwaydf[KEGGpathwaydf[comparison].isin(overlap)]['Symbol'])
    n_overlap = len(overlap) ##winnersfromA
    n_module = len(module_df[comparison]) ## teamA
    n_parkinsons = len(KEGGpathwaydf[comparison]) #totalwinners
    n_total = len(all_modules[comparison]) # total runners
    fold_enrichment = (n_overlap / n_module) / (n_parkinsons / 30702)
    ## probability overlap num is significant. is it more overlap than ranodom chance?
    ## https://www.youtube.com/watch?v=Go3qssiBzuY
    probability = hypergeom.pmf(n_overlap, 30702, n_parkinsons, n_module)
    # print(probability)
    p_value = 1- hypergeom.cdf(n_overlap, 30702, n_parkinsons, n_module) ## 30702 genes in mouse
    return module_num, n_module, fold_enrichment, probability, p_value, overlap

DIY_enrichment_df = pd.DataFrame(columns=['Module Number', '# Genes in Module',
                                          'Fold Enrichment', 'Probability', 'P-Value', 'Overlap Genes']
                                 )
for member in range(0, max(all_modules['Membership'].unique())+1):
    my_mod = all_modules[all_modules['Membership'] == member]
    mod_num, n_genes_module, fold_enrich, prob, p_val, overlap = get_EnrichmentAnalysis(member, my_mod, all_modules, PDDF, 
                                                                        comparison='Entrez') ## comparison can be entrez,KeggOrtholog
    
    temp_df = pd.DataFrame([{'Module Number': mod_num, '# Genes in Module': n_genes_module,
                             'Fold Enrichment': fold_enrich, 
                             'Probability': prob, 'P-Value': p_val, 'Overlap Genes':overlap}])
    DIY_enrichment_df = pd.concat([DIY_enrichment_df, temp_df], ignore_index=True)

DIY_enrichment_df

,Module Number,# Genes in Module,Fold Enrichment,Probability,P-Value,Overlap Genes
0,0,31,3.723260,2.070497e-01,2.948265e-02,[Psma1]
1,1,144,20.038377,2.190781e-25,0.000000e+00,"[Ndufv1, Ndufa1, Ndufa4, Ndufa9, Ndufa10, Nduf..."
2,2,133,2.603482,8.035754e-02,2.881297e-02,"[Rps27a, Psmd3, Psmd4]"
3,3,99,5.829346,1.505310e-03,2.284244e-04,"[Gnal, Gnas, Gnai1, Adcy5, Itpr1]"
4,4,92,0.000000,4.485440e-01,5.514560e-01,[]
5,5,86,1.342105,3.562480e-01,1.710957e-01,[Eif2s1]
6,6,83,6.953075,7.007974e-04,8.634426e-05,"[Uba1, Uchl1, Cox4i1, Vdac1, Park7]"
7,7,80,10.099342,5.790949e-06,4.833199e-07,"[Psmc1, Psmc4, Psmc3, Camk2d, Camk2g, Camk2b, ..."
8,8,80,8.656579,6.397162e-05,6.274269e-06,"[Snca, Mapt, Th, Slc6a3, COX2, Kif5c]"
9,9,78,5.919028,4.164467e-03,5.927513e-04,"[Prkaca, Psmd11, Psmd6, Psmd14]"


In [15]:
SAVE_DIY_FILE = f"enrichments_DIY/{KOGENE}_{PATH_OF_INT}_hypergeometricDIY.csv"
ensure_dirs_exists(SAVE_DIY_FILE)
DIY_enrichment_df.to_csv(SAVE_DIY_FILE)


In [17]:
def gseapy_analysis(module_num, module_df, comparison='Symbol'):
    gene_list = list(module_df[comparison])
    # Custom Parkinson's disease-associated gene set
    # Perform enrichment analysis
    enr = gp.enrichr(gene_list=list(gene_list),
                            gene_sets=["KEGG_2019_Mouse", 'HDSigDB_Mouse_2021',
                                       'KOMP2_Mouse_Phenotypes_2022', 'Mouse_Gene_Atlas'],
                            outdir= None) #'plots')  # Set to None if you don't want to save results to disk
    enr_df = enr.results
    parkinson_disease_rows = enr_df[enr_df['Term'].str.lower().str.contains("parkinson", na=False)]
    parkinson_disease_rows.insert(0, 'Module_Number', module_num)
    parkinson_disease_rows.insert(1, "# Genes in Module", len(module_df[comparison]))
    return parkinson_disease_rows

GSEA_enrichment_df = pd.DataFrame()
for member in tqdm(range(0, max(all_modules['Membership'].unique())+1)):
    my_mod = all_modules[all_modules['Membership'] == member]
    res = gseapy_analysis(member, my_mod, comparison='Symbol') ## comparison can be entrez,KeggOrtholog
    GSEA_enrichment_df = pd.concat([GSEA_enrichment_df, res], ignore_index=True)

100%|██████████| 46/46 [02:07<00:00,  2.78s/it]


In [18]:
GSEA_enrichment_df

,Module_Number,# Genes in Module,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,0,31,HDSigDB_Mouse_2021,Genes Down-Regulated In Parkinson Disease Mark...,4/1054,7.805602e-02,3.319317e-01,0,0,2.669347,6.807713,ARHGAP44;OPHN1;OCRL;RGS7
1,0,31,HDSigDB_Mouse_2021,Parkinson Disease Markers In Human Substantia ...,5/2327,2.897202e-01,5.333807e-01,0,0,1.461522,1.810591,ARHGAP44;OPHN1;OCRL;RAF1;RGS7
2,0,31,HDSigDB_Mouse_2021,Genes Up-Regulated In Parkinson Disease Marker...,1/1278,8.710849e-01,9.097188e-01,0,0,0.487914,0.067340,RAF1
3,1,144,KEGG_2019_Mouse,Parkinson disease,22/144,3.307490e-23,2.629455e-21,0,0,29.168772,1509.870936,NDUFA9;NDUFA12;ATP8;NDUFA4;NDUFA10;ATP5A1;NDUF...
4,1,144,HDSigDB_Mouse_2021,Genes Down-Regulated In Parkinson Disease Mark...,26/1054,3.218780e-08,6.553586e-07,0,0,4.035547,69.619960,ATP5A1;NDUFA10;SLC2A3;ATP5O;HK1;ATP5B;OPA1;OXC...
...,...,...,...,...,...,...,...,...,...,...,...,...
121,43,6,HDSigDB_Mouse_2021,Parkinson Disease Markers In Human Substantia ...,3/2327,2.397741e-02,3.035323e-01,0,0,7.603270,28.365087,SURF4;PLCL2;WASF3
122,43,6,HDSigDB_Mouse_2021,Genes Down-Regulated In Parkinson Disease Mark...,2/1054,3.611859e-02,3.290727e-01,0,0,9.002852,29.897998,PLCL2;WASF3
123,43,6,HDSigDB_Mouse_2021,Genes Up-Regulated In Parkinson Disease Marker...,1/1278,3.271604e-01,5.845000e-01,0,0,2.931402,3.275269,SURF4
124,45,5,HDSigDB_Mouse_2021,Genes Down-Regulated In Parkinson Disease Mark...,1/1054,2.371734e-01,4.746295e-01,0,0,4.497151,6.471238,ATXN10


In [27]:
SAVE_GSEA_FILE = f"enrichments_GSEA/{KOGENE}_{PATH_OF_INT}_GSEApy.csv"
ensure_dirs_exists(SAVE_GSEA_FILE)
GSEA_enrichment_df.to_csv(SAVE_GSEA_FILE)


In [30]:
un = GSEA_enrichment_df['Term'].unique()
parkinson_disease_rows = GSEA_enrichment_df[GSEA_enrichment_df['Term'].str.contains("Parkinson", na=False)]

# Display the filtered rows
(parkinson_disease_rows)

,Module_Number,# Genes in Module,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,0,31,HDSigDB_Mouse_2021,Genes Down-Regulated In Parkinson Disease Mark...,4/1054,7.805602e-02,3.319317e-01,0,0,2.669347,6.807713,ARHGAP44;OPHN1;OCRL;RGS7
1,0,31,HDSigDB_Mouse_2021,Parkinson Disease Markers In Human Substantia ...,5/2327,2.897202e-01,5.333807e-01,0,0,1.461522,1.810591,ARHGAP44;OPHN1;OCRL;RAF1;RGS7
2,0,31,HDSigDB_Mouse_2021,Genes Up-Regulated In Parkinson Disease Marker...,1/1278,8.710849e-01,9.097188e-01,0,0,0.487914,0.067340,RAF1
3,1,144,KEGG_2019_Mouse,Parkinson disease,22/144,3.307490e-23,2.629455e-21,0,0,29.168772,1509.870936,NDUFA9;NDUFA12;ATP8;NDUFA4;NDUFA10;ATP5A1;NDUF...
4,1,144,HDSigDB_Mouse_2021,Genes Down-Regulated In Parkinson Disease Mark...,26/1054,3.218780e-08,6.553586e-07,0,0,4.035547,69.619960,ATP5A1;NDUFA10;SLC2A3;ATP5O;HK1;ATP5B;OPA1;OXC...
...,...,...,...,...,...,...,...,...,...,...,...,...
121,43,6,HDSigDB_Mouse_2021,Parkinson Disease Markers In Human Substantia ...,3/2327,2.397741e-02,3.035323e-01,0,0,7.603270,28.365087,SURF4;PLCL2;WASF3
122,43,6,HDSigDB_Mouse_2021,Genes Down-Regulated In Parkinson Disease Mark...,2/1054,3.611859e-02,3.290727e-01,0,0,9.002852,29.897998,PLCL2;WASF3
123,43,6,HDSigDB_Mouse_2021,Genes Up-Regulated In Parkinson Disease Marker...,1/1278,3.271604e-01,5.845000e-01,0,0,2.931402,3.275269,SURF4
124,45,5,HDSigDB_Mouse_2021,Genes Down-Regulated In Parkinson Disease Mark...,1/1054,2.371734e-01,4.746295e-01,0,0,4.497151,6.471238,ATXN10


In [31]:
gene_set_libraries = gp.get_library_name()
for x in gene_set_libraries:
    if "mouse" in x.lower():
        print(x)
gene_set_mouse = "KEGG_2019_Mouse"

HDSigDB_Mouse_2021
KEGG_2019_Mouse
KOMP2_Mouse_Phenotypes_2022
Mouse_Gene_Atlas
RNAseq_Automatic_GEO_Signatures_Mouse_Down
RNAseq_Automatic_GEO_Signatures_Mouse_Up
WikiPathways_2019_Mouse


Some mouse libraries: HDSigDB_Mouse_2021
KEGG_2019_Mouse
KOMP2_Mouse_Phenotypes_2022
Mouse_Gene_Atlas
RNAseq_Automatic_GEO_Signatures_Mouse_Down
RNAseq_Automatic_GEO_Signatures_Mouse_Up
WikiPathways_2019_Mouse